# Modelamiento

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357282 sha256=8537b67ae04e44a7da7f41cb56c16ea10fad8904ab8c5342755a4bec3dbdb7e7
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [5]:
!pip install dask-ml

INFO: pip is looking at multiple versions of distributed to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.8/149.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.3/237.3 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.2 MB/s eta 0:00:00


In [6]:
# Imports
import dask.dataframe as dd
import pandas as pd
import numpy as np
from surprise import accuracy, Dataset, Reader
from surprise.model_selection import train_test_split, GridSearchCV
from surprise.prediction_algorithms.knns import KNNBasic
from collections import defaultdict
from surprise.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
datos = dd.read_parquet('/content/drive/MyDrive/EAFIT/Semestre 3/Mineria de grandes Volumenes de Datos/Proyecto Integrador/data1pct.parquet',  engine="pyarrow")
datos.head()

,customer_id,product_id,star_rating
0,24912421,B000A3WS84,5.0
1,22668859,B0032QA93M,5.0
2,29511033,B005HB8JN2,1.0
3,35207047,B00BQ5RYI4,5.0
4,20204904,B00AWJEOFG,5.0


In [8]:
len(datos) # corresponde al 10% del dataset

1098284

In [9]:

# Muestrear de nuevo  al % del DataFrame
data_model = datos.sample(frac=0.03, random_state=42)

In [10]:
# Convertirlo a un Pandas DataFrame para operar localmente
data_model = data_model.compute()

# Verificar información del DataFrame
data_model.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32951 entries, 2482 to 3
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   customer_id  32951 non-null  int64  
 1   product_id   32951 non-null  string 
 2   star_rating  32951 non-null  float64
dtypes: float64(1), int64(1), string(1)
memory usage: 1.3 MB


In [12]:
from surprise import Reader, Dataset

# Crear un objeto Reader con el rango de calificaciones
reader = Reader(rating_scale=(1, 5))  # Calificaciones de 1 a 5

# Cargar datos desde el DataFrame
#sdata = Dataset.load_from_df(data_model[['customer_id', 'product_id', 'avg_star_rating']], reader)
sdata = Dataset.load_from_df(data_model[['customer_id', 'product_id', 'star_rating']], reader)


In [13]:
# Dividir en entrenamiento y prueba
trainset, testset = train_test_split(sdata, test_size=0.2, random_state=42)

In [14]:
# Configurar similitud para KNNBasic
sim_options = {'name': 'pearson', 'user_based': False}
sim_item_item = KNNBasic(sim_options=sim_options, verbose=False)

In [15]:
# Entrenar el modelo
sim_item_item.fit(trainset)

In [16]:
# Métricas de evaluación
def precision_recall_at_k(model, testset, k=10, threshold=3.5):
    """Calcula precisión y recall en k para el modelo."""
    predictions = model.test(testset)
    user_est_true = defaultdict(list)

    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold)) for (est, true_r) in user_ratings[:k])
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    precision = round(sum(prec for prec in precisions.values()) / len(precisions), 3)
    recall = round(sum(rec for rec in recalls.values()) / len(recalls), 3)
    f1_score = round(2 * precision * recall / (precision + recall) if (precision + recall) != 0 else 0, 3)

    accuracy.rmse(predictions)
    print(f'Precision: {precision}, Recall: {recall}, F1 Score: {f1_score}')

In [17]:
# Evaluar el modelo
precision_recall_at_k(sim_item_item, testset)

RMSE: 1.2751
Precision: 0.782, Recall: 0.782, F1 Score: 0.782


In [18]:
# Búsqueda de hiperparámetros
param_grid = {
    'k': [5, 10],
    'min_k': [1, 5],
    'sim_options': {'name': ['pearson', 'cosine'], 'user_based': [False]}
}

gs = GridSearchCV(KNNBasic, param_grid, measures=['rmse'], cv=3, n_jobs=-1)
gs.fit(sdata)

print("Mejor RMSE:", gs.best_score['rmse'])
print("Mejores parámetros:", gs.best_params['rmse'])

Mejor RMSE: 1.2851752893145003
Mejores parámetros: {'k': 5, 'min_k': 1, 'sim_options': {'name': 'pearson', 'user_based': False}}
